In [1]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode
import pandas as pd
from pyecharts.charts import Pie
from pyecharts.charts import EffectScatter
from pyecharts.globals import SymbolType
from pyecharts.charts import Scatter3D
from pyecharts.globals import ThemeType
from pyecharts.charts import Radar
from pyecharts.charts import Line

### 加载当前文件夹下的excel

In [2]:
df = pd.read_excel('glory_new.xlsx')

### 柱状图-英雄定位/数量

In [4]:
def bar_border_radius():
    position_name = list(df['position'].value_counts().keys()) # 读取 position 列，分组计数，以定位分组
    position_value = [v for k,v in df['position'].value_counts().items()] # 统计具体个数
    c = (
        Bar()
        .add_xaxis(position_name)
        .add_yaxis("英雄定位", position_value, category_gap="60%")
        .set_series_opts(itemstyle_opts={
            "normal": {
                "color": JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                    offset: 0,
                    color: 'rgba(0, 244, 255, 1)'
                }, {
                    offset: 1,
                    color: 'rgba(0, 77, 167, 1)'
                }], false)"""),
                "barBorderRadius": [30, 30, 30, 30],
                "shadowColor": 'rgb(0, 160, 221)',
            }})
        .set_global_opts(title_opts=opts.TitleOpts(title="柱状图-英雄定位/数量"))
    )
    return c

bar_border_radius().render_notebook() # 初始化运行在notebook上。

### 饼图/英雄定位

In [5]:
def pie_base() -> Pie:
    position_name = list(df['position'].value_counts().keys()) # 读取 position 列，分组计数，以定位分组
    position_value = [v for k,v in df['position'].value_counts().items()] # 统计具体个数
    c = (
        Pie()
        .add("", [list(z) for z in zip(position_name, position_value)])
        .set_global_opts(title_opts=opts.TitleOpts(title="饼图/英雄定位"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
    return c
pie_base().render_notebook()

### 涟漪特效散点图-英雄移动速度

In [6]:
def effectscatter_base() -> EffectScatter:
    c = (
        EffectScatter()
        .add_xaxis(list(df['name']))
        .add_yaxis("", list(df['移速']))
        .set_global_opts(title_opts=opts.TitleOpts(title="涟漪特效散点图-英雄移动速度"))
    )
    return c
effectscatter_base().render_notebook()

### 柱状图-英雄/最大生命值

In [7]:
def bar_base_dict_config() -> Bar:
    tk_name = list(df[df['position']=='坦克']['name']) # 获取等于坦克的英雄名称
    tk_list = list(df[df['position']=='坦克']['最大生命']) # 获取等于坦克的最大生命
    c = (
        Bar({"theme": ThemeType.MACARONS,"width": "1024px", "height": "768px"})
        .add_xaxis(tk_name)
        .add_yaxis("坦克", tk_list, category_gap="20%")
        .set_global_opts(title_opts=opts.TitleOpts(title="柱状图-英雄/最大生命值"),datazoom_opts=opts.DataZoomOpts())
    )
    return c
bar_base_dict_config().render_notebook()

### 坦克生命值平均值

In [8]:
def bar_markline_custom() -> Bar:
    x_data = list(df[df['position']=='坦克']['name'])
    y_data = list(df[df['position']=='坦克']['最大生命'])
    y_mean = df['最大生命'].mean()

    c = (
        Bar()
        .add_xaxis(x_data)
        .add_yaxis("生命值", y_data)
        .set_global_opts(title_opts=opts.TitleOpts(title="英雄/生命值"),datazoom_opts=opts.DataZoomOpts())
        .set_series_opts(
            itemstyle_opts={
            "normal": {
                "color": JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                    offset: 0,
                    color: 'rgba(221,160,221, 1)'
                }, {
                    offset: 1,
                    color: 'rgba(255,0,255, 1)'
                }], false)"""),
                "barBorderRadius": [30, 30, 30, 30],
                "shadowColor": 'rgb(0, 160, 221)',
            }},
            label_opts=opts.LabelOpts(is_show=False),
            markline_opts=opts.MarkLineOpts(
                data=[opts.MarkLineItem(y=y_mean, name="yAxis=50",type_='average')]
            ),
        )
    )
    return c
bar_markline_custom().render_notebook()

### 柱状图-英雄/防御值

In [9]:
def bar_base() -> Bar:
    c = (
        Bar({"width": "1024px", "height": "768px"})
        .add_xaxis(list(df['name']))
        .add_yaxis("物理防御",list(df['物理防御']),color=Faker.rand_color())
        .set_global_opts(title_opts=opts.TitleOpts(title="柱状图-英雄/防御值"),datazoom_opts=opts.DataZoomOpts())
    )
    return c
bar_base().render_notebook()

### 雷达图-英雄推荐

In [10]:
def radar_base(name=None) -> Radar:
    # 如果外面传入职位，则直接替换
    if name:
        new_df = df[df['position']==name]
    else:
        new_df = df
        name = '英雄'
    data = [[a,b,c,d,e] for a,b,c,d,e in zip(list(new_df['生存能力']), list(new_df['攻击伤害']),
                                         list(new_df['技能效果']), list(new_df['上手难度']),
                                         list(new_df['name']))]
    
    c = (
        Radar()
        .add_schema(
            schema=[
                opts.RadarIndicatorItem(name="生存能力", max_=10),
                opts.RadarIndicatorItem(name="攻击伤害", max_=10),
                opts.RadarIndicatorItem(name="技能效果", max_=10),
                opts.RadarIndicatorItem(name="上手难度", max_=10),
                opts.RadarIndicatorItem(name="英雄名称", max_=10)
            ]
            , shape="circle",
            
        )
        .add(name, data)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="雷达图-英雄推荐"))
    )
    return c
print(list(df['position'].value_counts().keys())) # 打印全位置
radar_base('刺客').render_notebook()

['法师', '战士', '坦克', '射手', '辅助', '刺客']
